In [1]:
%load_ext py_d3

In [37]:
%%d3 4.4.4

<svg width=800 height=400></svg>

<script>
    requirejs.config({
        paths: {
            tree: "https://unpkg.com/threshold-tree@1.0.3/umd/threshold-tree"
        }
    });
    
    requirejs(['tree'], function(tree) {
        
        d3.csv("../data/complaint_types.csv", function(complaints) {

            // Build the threshold tree and convert it to a simple JSON format.
            let hr = new tree.ThresholdTree(complaints).as_hierarchy();
            
            // Convert that to a hierarchy.
            console.log(d3.stratify().id(d => d.name).parentId(d => d.parent)(hr));
            let apple = d3.stratify().id(d => d.name).parentId(d => d.parent)(hr)
            
            // Convert the input to a D3 hierarchy (data type shared by treemap, tree, etc.)
            // let root = d3.hierarchy(hr);
            
            console.log(apple);
            
            // Create our treemap layout factory function.
            let treemap = d3.treemap()
                .tile(d3.treemapResquarify)
                .size([800, 400])
                .round(true)
                .paddingInner(1);
            
            // Apply our treemap function to our data.
            // treemap(root);
            
            // Select the SVG.
            const svg = d3.select("svg");
            
            // Lay out the cells.
            let cell = svg.selectAll("g")
                .data(root.leaves())
                .enter()
                .append("g")
                .attr("transform", function(d) { return "translate(" + d.x0 + "," + d.y0 + ")"; });
            
            console.log(root);
            // console.log(cell);

            cell.append("rect")
                .attr("id", function(d) { return d.data.id; })
                .attr("width", function(d) { return d.x1 - d.x0; })
                .attr("height", function(d) { return d.y1 - d.y0; })
                .attr("fill", function(d) { return "transparent"; })
                .attr("stroke", function(d) { return "black"; })
                .attr("stroke-width", "1");
        });
        
    });
</script>